<h1 style="color:#D30982; text-align:center;">Variational Quantum Eigensolver</h1> 



<h1 style="color:#D30982;">Overview</h1>

- Overview of Variational Quantum Eigensolver (VQE)
- Implementing VQE for Finding the Ground State of a Hamiltonian
- Performance of Various Ansatz and Optimizers in VQAs

<h1 style="color:#D30982;">Overview of VQE</h1>

- The VQE algorithm was first proposed and demonstrated by Peruzzo et. al. in 2014. It is based on Rayleigh-Ritz method, which is an approximate method to find the lowest eigenvalue of a given Hamiltonian.
- Variational Quantum Eigensolver (VQE) is a hybrid quantum-classical algorithm
- Goal: Find the ground state energy (lowest eigenvalue) of a given Hamiltonian (H), which describes the system of interest in quantum mechanics

$$ E_{ground} = \text{min} \braket{\psi (\theta)| H | \psi(\theta)} \;.$$

- Procedure: Minimize the expectation value of the Hamiltonian with respect to a quantum state, which is a parameterized quantum circuit (ansatz)
- VQE combines the strengths of quantum computers (manipulating high-dimensional quantum states) and classical computers (performing optimization)

<h1 style="color:#D30982;">Relationship Between VQE and QAOA</h1>

- Quantum Approximate Optimization Algorithm (QAOA) and Variational Quantum Eigensolver (VQE) are both hybrid quantum-classical algorithms
- Both algorithms use parameterized quantum circuits (ansatz) and classical optimization techniques to solve their respective problems
- VQE focuses on finding the ground state energy of a given Hamiltonian, while QAOA targets combinatorial optimization problems by approximating the ground state of a problem-specific cost Hamiltonian

- QAOA's ansatz is derived from the structure of the problem-specific cost Hamiltonian
- QAOA circuits alternate between applying a problem-specific Hamiltonian (often referred to as the "cost" or "problem" Hamiltonian) and a reference Hamiltonian (typically referred to as the "mixer" Hamiltonian)
- This alternating structure is inspired by adiabatic quantum computing and the Trotter-Suzuki decomposition
- In contrast, VQE's ansatz is more general, as it can be designed to represent a wide range of quantum states and can be applied to various problems

<h2 style="color:#9A11DA;"> Designing and testing an ansatz </h2>

- Considerations:

    - Number of qubits
    
    - Connectivity between qubits
    
    - Gate set available on the hardware
    
    - Structure of the problem Hamiltonian

- Steps:

    - Define the quantum circuit structure.
    
    - Add parameterized single-qubit gates (e.g., RX, RY, RZ).
    
    - Add entangling gates (e.g., CNOT, CZ) to create correlations between qubits.
    
    - Repeat steps 2 and 3 to create deeper circuits if necessary.

- Metrics to consider:

    - Convergence speed: How quickly does the VQE algorithm find the ground state energy?

    - Ground state accuracy: How close is the VQE result to the true ground state energy?
    
    - Resource usage: How many qubits, gates, and optimization iterations are required?
    

- Tools for comparison:

    - Simulators: Compare the performance of different ansatz structures using classical simulators before running on real quantum hardware.
    
    - Noise models: Investigate the impact of noise on the performance of different ansatz structures.

Before we actually code the VQE algorithm and understand how it is a promising algorithm for noisy intermediate scale quantum (NISQ) devices, let's start begin with of a simple optimization problem and a classical algorithm to solve it.

Consider a simple function, $x^2$. We can find its minimum value by using various optimizers available in scipy package in python. 

In [ ]:
import numpy as np
from scipy.optimize import minimize

def parabola(x):
    return x**2

x1=10
res_nm = minimize(parabola, x1, method='nelder-mead',
            options={'xatol': 1e-8, 'disp': True})

In the above code, the python method parabola is the function we want to minimize and we use the minimize function availabe in the optimize module of scipy package. For this particular case, we use the 'Nelder-Mead' method which is a optimization technique that uses direct search method when the gradient information is not available. This is the reason that the algorithm evaluated the function many times before it found the minimum. With functions when the derivative is available, then other algorithms could be used which work much faster. e.g. the BFGS algorithm

In [ ]:
def parabola_derivative(x):
    return 2*x

res_bfgs = minimize(parabola, x1, method='BFGS', jac=parabola_derivative,
               options={'disp': True})

As can be seen above that, when the derivative of a function is available, the number of iterations as well as number of function evaluations are much lower. We can also try it with another function that takes in vectors, e.g. the Rosenbrock function, which is given as follows: 
$$f(x)=\Sigma_i^{N-1} 100(x_{i+1}-x_i^2)^2+(1-x_i)^2$$

In [ ]:
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)


import numpy
def torque(F,r):
    f_normalized = F/numpy.linalg.norm(F)
    r_normalized = F/numpy.linalg.norm(r)
    theta = numpy.arccos(numpy.dot(f_normalized,r_normalized))
    return numpy.linalg.norm(F)*numpy.linalg.norm(r)*numpy.sin(theta)


x1 = numpy.random.rand(15) #array([.3, 0.75, 1.5, 2.9, .2])
print(x1)
res_nm = minimize(rosen, x1, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

<h2 style="color:#9A11DA; text-align:center">  Quantum World!  </h2>  <a name="No_Installation"></a>
Now, let's get into the quantum world and consider a simple Hamiltonian of two spins given as 

$$H=Z\otimes Z$$

where $Z$ is a Pauli-Z matrix given by 
$$Z = \begin{pmatrix} 
1 & 0 \\
0 & -1
\end{pmatrix}
$$
and $\otimes$ is just the tensor product. So, the matrix representation of the Hamiltonian is as follows:
$$Z = \begin{pmatrix} 
1 & 0 &0 &0 \\
0 & -1 &0 &0 \\
0&0& -1 & 0 \\
0&0&0 & 1
\end{pmatrix}
$$

Given $H$, our job is to find its lowest eigenvalue. With the given Hamiltonian, we can just run the Gauss elimination process and obtain the lowest eigenvalue as follows:

In [ ]:
import numpy as np 
from qiskit.opflow import Z, StateFn
from qiskit.circuit.library import PauliTwoDesign
observable = Z ^ Z 
np.linalg.eigvalsh(observable.to_matrix())

From above, we can see that the lowest eigenvalue of the Hamiltonian is -1. Now, let's try to find the eigenvalue using variational method. After, we describe the variational method, we will present the motivation for using it.

In variational method, we prepare an ansatz/approximate eigenvector of the Hamiltonian which is parameterized. Then a cost-function is defined as the expectation value of this Hamiltonian. The ansatz can then be optimized using the optimization algorithms we used at the beginning of this blog post. In the code that follows, we demonstrate this procedure using IBM's qiskit library. Note that the way we parameterize the ansatz is using a quantum-circuit. We will comment later, why we approach the problem this way.


In [ ]:
import numpy as np
# The optimization algorithms in qiskit are very similar to scipy 
# and in some cases are just wrappers around scipy optimizers
from qiskit.algorithms.optimizers import SPSA, NELDER_MEAD, COBYLA
from qiskit.circuit.library import PauliTwoDesign
from qiskit.opflow import Z, StateFn

ansatz = PauliTwoDesign(2, reps=1, seed=1) # defining a quantum circuit for parameterized ansatz (* Exercise: Look up the documentation for PauliTwoDesign. Show that `\sqrt{H} = RY(\pi/4)` *)
observable = Z ^ Z # defining the Hamiltonian
# Random seed which defines "rotations" in the quantum circuit
initial_point = np.random.random(ansatz.num_parameters) 

# loss function is just the expectation value of the Hamiltonian 
def loss(x): 
    bound = ansatz.bind_parameters(x)
    return np.real((StateFn(observable, is_measurement=True) @ StateFn(bound)).eval()) 

# Draw the parameterized circuit.
ansatz.bind_parameters(initial_point).decompose().draw()

In [ ]:
# In this code block we get the statevector corresponding to the quantum circuit
# and the matrix form of the Hamiltonian that we defined.

from qiskit.quantum_info.states.quantum_state import QuantumState
from qiskit import circuit
from qiskit import Aer
import copy
# Run the quantum circuit on a statevector simulator backend
backend = Aer.get_backend('statevector_simulator')
x=copy.deepcopy(ansatz.bind_parameters(initial_point).decompose())
job = backend.run(x)
result = job.result()
outputstate = result.get_statevector(x, decimals=3)
print('The initial point is', initial_point,'\n')
print('Parameterized Eigenvector is', outputstate,'\n')

# Import Aer
print("The matrix form of the Hamiltonian is given by",observable.to_matrix())
print('The cost function value at the initial point is:',loss(initial_point))

#
#

Things to note here: 
- The Hamiltonian is the same matrix we diagonalized using the methods available in numpy. 
- The ansatz for the cost-function is an eigenvector which is parameterized. 
- The above code cell prints the starting guess for the eigenvector, the initial seed for the parameters, and the initial value of the cost function. 

In the following cell, we will run a minimization routine on the loss function that we defined above.

In [ ]:
# Running the optimization using SPSA, COBYLA and Nelder-Mead
spsa = SPSA(maxiter=500)
cobyla = COBYLA(maxiter=300)
nm = NELDER_MEAD(maxiter=300)

result_spsa = spsa.minimize(loss, x0=initial_point)
result_nm = nm.minimize(loss, x0=initial_point)
result_cobyla = cobyla.minimize(loss, x0=initial_point)

In [ ]:
print('The value of x for the minimum value is: ',result_spsa.x)
print('The lowest eigenvalue of Hamiltonian is: ',result_spsa.fun)

print('The value of x for the minimum value is: ',result_nm.x)
print('The lowest eigenvalue of Hamiltonian is: ',result_nm.fun)

print('The value of x for the minimum value is: ',result_cobyla.x)
print('The lowest eigenvalue of Hamiltonian is: ',result_cobyla.fun)

#

It can be checked that the the optimization procedure does work and the final value of the cost-function after the optimization is close to the actual value. To get better results, we can perhaps increase the maximum number of iterations, but you will see that it does not help much. The main problem here is that the ansatz is not very good.

<h2 style="color:#9A11DA; text-align:center">  Understanding structure of Ansatz!  </h2>  <a name="No_Installation"></a>

The PauliTwoDesign ansatz, that we used, is constructed by applying an initial layer of $\pi/4$ Pauli-Y rotations followed by alternating layers of single-qubit Pauli rotation gates on each of the qubits and nearest-neighbor CZ gates with total depth of two. The rotation angle of each of the Pauli-rotation gates is a parameter that can be tuned.  A single-layer of PauliTwoDesign consists of a layer of CZ gates sandwiched between two layers of single qubit Pauli gates. With this setup, it allows us to span a relatively small subspace of the Hilbert space associated with qubits. And this is the reason the minimum value from optimization is not close to the global minimum of the cost function. So, we can increase the depth of the circuit and add more CNOT gates to increase the entanglement in the system. This will span a relatively larger subspace and provide us with a greater chance to find the minimum.

In [ ]:
ansatz = PauliTwoDesign(2, reps=2, seed=1) # defining a quantum circuit for parameterized ansatz
# Random seed which defines "rotations" in the quantum circuit
initial_point = np.random.random(ansatz.num_parameters) 
# Draw the parameterized circuit.
ansatz.bind_parameters(initial_point).decompose().draw()

In [ ]:
# Running the optimization using SPSA, COBYLA and Nelder-Mead
spsa = SPSA(maxiter=500)
cobyla = COBYLA(maxiter=300)
nm = NELDER_MEAD(maxiter=300)

result_spsa = spsa.minimize(loss, x0=initial_point)
result_nm = nm.minimize(loss, x0=initial_point)
result_cobyla = cobyla.minimize(loss, x0=initial_point)

In [ ]:
print('The value of x for the minimum value is: ',result_spsa.x)
print('The lowest eigenvalue of Hamiltonian is: ',result_spsa.fun)

print('The value of x for the minimum value is: ',result_nm.x)
print('The lowest eigenvalue of Hamiltonian is: ',result_nm.fun)

print('The value of x for the minimum value is: ',result_cobyla.x)
print('The lowest eigenvalue of Hamiltonian is: ',result_cobyla.fun)

#

As expected, with greater entanglement in the system the value of the cost-function converges to the actual lowest eigenvalue. **We want to emphasize that there is nothing special about the PauliTwoDesign ansatz.** We are allowed to use any ansatz that we like.  For this problem, PauliTwoDesign gave us good results, but there is no reason, why it should give us good resutls for problems in quantum machine learning, quantum chemistry or any other problem for that matter. In fact, choosing the best ansatz is an art that people are still learning. Generally, the best ansatz are rooted in some intuition from the field in which the cost function originates. For example, the UCCSD ansazt works well for quantum chemistry problems.

<h2 style="color:#9A11DA; text-align:center">  Motivation for Variational method!  </h2>  <a name="No_Installation"></a>

Now, that we have shown that the two methods, the exact diagonalization and VQE method, output the same result, we want to comment on when would we want to choose one method over the other. The gaussian elimination's time complexity is $O(2n^3/6)$ (*for reference: https://www.ndsu.edu/pubweb/~novozhil/Teaching/488%20Data/08.pdf *), where, n, is the size of the matrix. The variational method performs better, compared to exact diagonalization, when the matrix to be diagonalized is very large and there is a good estimate for the ansatz available. With a good ansatz, the computational resources required for VQE may be much less than the Gaussian elimination process.

Let us now demonstrate a quantum model where such a situation occurs. In the Ising model, as the number of spins in the systems grow, the size of the Hamiltonian grows exponentially. This means that, except for a few special cases in lower dimensional systems, the Hamiltonian cannot be diagonalized exactly, even for tens of spins in the system.

<h2 style="color:#9A11DA; text-align:center">  Ising model and NP completeness!  </h2>  <a name="No_Installation"></a>
Now, let us consider the Ising model. It has been one of the most studied quantum systems. It consists of interacting quantum spins (two-level systems), arranged on a graph. This simplified model was originally proposed to study magnetism, but has been found surprisingly useful to model many other physical systems, like lattice gas and spin glass. Furthermore, an ising model on a nonplanar lattice has been shown to be NP-complete. An efficient method has been devised to map one of the most important problems in computer science, max-cut, to the Ising model. Given the importance of the model, it is a good candidate to study using VQE.

The Hamiltonian for the Ising model is given as follows
$$H(\sigma) = \Sigma_{(i,j)} J_{ij}\sigma_i\sigma_j -\mu \Sigma_j h_j \sigma_j, \quad i,j \in \mathcal{E}, \quad \sigma_j \in \{-1,+1\}$$

where, $\mathcal{E}$ is the set of edges (nearest-neighbors), $J_{ij}$ is the interaction energy between two spins, $\mu$ is the magnetic moment of a spin and $h_j$ is the magnetic field interacting with the spin. 


For our VQE model, we can consider a small system of only four spins with the following Hamiltonian:

$$H(\sigma) = -J_{12}\sigma_1\sigma_2 -J_{23}\sigma_1\sigma_2 -J_{34}\sigma_3\sigma_4-\mu h_1 \sigma_1-\mu h_2 \sigma_2-\mu h_2 \sigma_3-\mu h_4 \sigma_4$$
We can even simplify it further by getting rid of the magnetic interaction term:
$$H(\sigma) = -J_{12}\sigma_1\sigma_2 -J_{23}\sigma_1\sigma_2 -J_{34}\sigma_3\sigma_4$$ 

In [ ]:
import numpy as np
import pylab
from IPython.display import clear_output
from qiskit import Aer
from qiskit.opflow import X, Z, I,Y
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SLSQP
from qiskit.circuit.library import TwoLocal

# defining the Hamiltonian
H2_op = (-1.052373245772859 * Z ^ Z ^ I^ I) + \
        (0.39793742484318045 * I ^ Z^ Z^ I) + \
        (-0.39793742484318045 * I ^ I^ Z^ Z) 

# (-0.01128010425623538 * Z ^ Z^ I^ X) + \
#         (-0.11128010425623538 * Y ^ Z^ I^ X) + \
#         (-0.05128010425623538 * Z ^ Y^ I^ X) + \
#         (0.18093119978423156 * X ^ X^ I^ Z)

# Using three different optimzers
optimizers = [COBYLA(maxiter=80), L_BFGS_B(maxiter=60), SLSQP(maxiter=60)]
converge_cnts = np.empty([len(optimizers)], dtype=object)
converge_vals = np.empty([len(optimizers)], dtype=object)

for i, optimizer in enumerate(optimizers):
    print('\rOptimizer: {}        '.format(type(optimizer).__name__), end='')
    algorithm_globals.random_seed = 50
    ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')

    counts = []
    values = []
    def store_intermediate_result(eval_count, parameters, mean, std):
        print(mean)
        clear_output()
        counts.append(eval_count)
        values.append(mean)

    vqe = VQE(ansatz, optimizer, callback=store_intermediate_result,
              quantum_instance=QuantumInstance(backend=Aer.get_backend('statevector_simulator')))
    result = vqe.compute_minimum_eigenvalue(operator=H2_op)
    converge_cnts[i] = np.asarray(counts)
    converge_vals[i] = np.asarray(values)
print('\rOptimization complete      ');
print('The minimum values found by the three optimization procedures are:', converge_vals[0][len(converge_vals[0])-1],', ',converge_vals[1][len(converge_vals[0])-1],', ',converge_vals[2][len(converge_vals[0])-1])


#

The above code shows how to get the minimum energy of the Hamiltonian using VQE. This procedure is the same as the procedure we used for smaller problems. But, here we can easily observe how the exact diagonalization does not scale well as the problem size grows. For our four spin system, the size of the matrix to be diagonalized is $16\times 16$. For a system with $N$-spins, the size of the matrix to be diagonalized is $2^N\times 2^N$, which for a few hundred spins is greater than the number of particles in the universe.

This is why running VQE with a good ansatz is the best bet. As we saw in our first example, it is not gauranteed that it would converge to the global minimum if the subspace spanned by the ansatz generated by all the values of parameters, does not span the space in which the global minimum lies. This is where we can use knowledge from physics to design better ansatz and possibly find the global minimum.
